**DATA PREPARATION**

In [2]:
import os
from pickle import load, dump
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical, plot_model
import matplotlib.pyplot as plt
from keras.models import Model
from keras.layers import Input, Dense, LSTM, Embedding, Dropout
from keras.layers import Add
from keras.callbacks import ModelCheckpoint
import string
import numpy as np


def extract_features(directory):
    model = VGG16()
    model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
    print(model.summary())

    features = {}
    for name in os.listdir(directory):
        filename = directory + '\\' + name
        image = load_img(filename, target_size=(224, 224))
        image = img_to_array(image)
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
        image = preprocess_input(image)
        feature = model.predict(image, verbose=0)
        image_id = name.split('.')[0]
        features[image_id] = feature
        # print('>', name)
    return features


dataset_img = "..\\train data\\images"
features = extract_features(dataset_img)
print('Extracted features: ', len(features))
dump(features, open('features.p', 'wb'))

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [3]:
def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

dataset_image_path = '..\\train data\\'
doc = load_doc(dataset_image_path + "captions.txt")
# print(doc.split('\n'))

*(Необязательно)*

In [4]:
def dump_imgIDs(doc, filename, start, end):
    imgIDs_list = []
    for line in doc.split('\n')[start:end]:
        imgID = line.split(",")[0]
        imgIDs_list.append(imgID)
    imgIDs = "\n".join(imgIDs_list)
    file = open(filename, 'w')
    file.write(imgIDs)
    
dump_imgIDs(doc, "trainImages.txt", 0, 30000)
dump_imgIDs(doc, "testImages.txt", 30000, 40455)

In [5]:
def load_descriptions(doc):
    mapping = {}
    for line in doc.split('\n'):
        tokens = line.split('.')
        if len(tokens) < 2:
            continue
        image_filename, image_desc = tokens[0], tokens[1:]
        image_id = image_filename.split('.')[0]
        image_desc = ''.join(image_desc)
        if image_id not in mapping:
            mapping[image_id] = []
        mapping[image_id].append(image_desc)
    return mapping

descriptions = load_descriptions(doc)
print('Loaded: ', len(descriptions))
# print(descriptions)

Loaded:  8091


In [6]:
def clean_descriptions(descriptions):
    table = str.maketrans('', '', string.punctuation)
    for img, cap in descriptions.items():
        for num_part, caption_part in enumerate(cap):

            caption_part.replace('-', ' ')
            desc = caption_part.split()

            desc = [word.lower() for word in desc]
            desc = [word.translate(table) for word in desc]
            desc = [word for word in desc if(len(word) > 1)]
            desc = [word for word in desc if(word.isalpha())]

            caption_part = ' '.join(desc)
            descriptions[img][num_part] = caption_part

    return descriptions

descriptions = clean_descriptions(descriptions)
# print(descriptions)

In [7]:
def to_vocabulary(descriptions):
    all_desc = set()
    for value in descriptions.values():
        [all_desc.update(d.split()) for d in value]
    return all_desc

vocab = to_vocabulary(descriptions)
# print(vocab)
print('Vocabulary size:', len(vocab))

Vocabulary size: 9287


In [8]:
def save_descriptions(descriptions, filename):
    lines = []
    for image_id, image_desc in descriptions.items():
        for desc in image_desc:
            lines.append(image_id + '\t' + desc)
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()
    
save_descriptions(descriptions, 'descriptions.txt')

**DEVELOPING DEEP LEARNING MODEL**

**Loading Data**

In [9]:
def load_set(filename):
    doc = load_doc(filename)
    dataset = []
    for line in doc.split('\n'):
        if len(line) < 1:
            continue
        identifier = line.split('.')[0]
        dataset.append(identifier)
    return set(dataset)

# trainImages = dataset + 'trainImages(test).txt'
train = load_set('trainImages.txt')
print('Training images loaded:', len(train))
print(train)

Training images loaded: 6000
{'2229177914_3308fe7d20', '3251976937_20625dc2b8', '3500829879_a643818d84', '3050606344_af711c726c', '241346709_23204cc2bc', '2279980395_989d48ae72', '3312779887_7682db7827', '2366421102_2d60d53a0e', '2836864045_9a093cfd65', '3401548798_3a93f2caa5', '2191329761_3effd856c5', '3015891201_2c1a9e5cd7', '2646046871_c3a5dbb971', '3164347907_2813f8ff0b', '2317714088_bcd081f926', '2226534154_cbcab7ba32', '2808870080_4ea4f3327e', '2869491449_1041485a6b', '3372215826_b3e6403b2e', '3211556865_d1d9becf69', '2188192752_09d9fc5431', '123889082_d3751e0350', '3330007895_78303e8a40', '3501781809_88429e3b83', '2944836001_b38b516286', '3275704430_a75828048f', '242559369_9ae90ed0b4', '2518853257_02f30e282e', '2067362863_59577f9d4d', '2278776373_fe499a93be', '255330891_86d65dfdbf', '2987096101_a41896187a', '1410193619_13fff6c875', '241347760_d44c8d3a01', '2834103050_512e5b330a', '3454315016_f1e30d4676', '2994205788_f8b3f2e840', '3036382555_30b7312cf3', '3155365418_43df5486f9', 

In [10]:
def load_clean_descriptions(filename, dataset):
    doc = load_doc(filename)
    descriptions = {}
    for line in doc.split('\n'):
        tokens = line.split('\t')
        image_filename, image_desc = tokens[0], tokens[1:]
        image_id = image_filename.split('.')[0]
        if image_id in dataset:
            if image_id not in descriptions:
                descriptions[image_id] = []
            desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
            descriptions[image_id].append(desc)
    return descriptions

train_descriptions = load_clean_descriptions('descriptions.txt', train)
print("Training descriptions loaded:", len(train_descriptions))
# print(train_descriptions)

Training descriptions loaded: 6000


In [11]:
def load_photo_features(filename, dataset):
    all_features = load(open(filename, 'rb'))
    features = {k: all_features[k] for k in dataset}
    return features

train_features = load_photo_features('features.p', train)
print('Features loaded:', len(train_features.keys()))
# for i, feature in enumerate(features.values()):
#     print(f'@{i} - {feature}')

Features loaded: 6000


In [12]:
def dict_to_list(descriptions):
    all_desc = []
    for values in descriptions.values():
        [all_desc.append(x) for x in values]
    return all_desc

def create_tokenizer(descriptions):
    lines = dict_to_list(descriptions)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

tokenizer = create_tokenizer(train_descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary size:', vocab_size)
dump(tokenizer, open('tokenizer.p', 'wb'))

Vocabulary size: 8109


*Вариант 1*

In [13]:
def create_sequences(tokenizer, max_length, descriptions, photos, vocab_size):
    x1, x2, y = list(), list(), list()
    # walk through each image identifier
    for key, desc_list in descriptions.items():
        # walk through each description for the image
        for desc in desc_list:
            # encode the sequence
            seq = tokenizer.texts_to_sequences([desc])[0]
            # split one sequence into multiple X,y pairs
            for i in range(1, len(seq)):
                # split into input and output pair
                in_seq, out_seq = seq[:i], seq[i]
                # pad input sequence
                in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                # encode output sequence
                out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                # store
                x1.append(photos[key][0])
                x2.append(in_seq)
                y.append(out_seq)
    return array(x1), array(x2), array(y)

def max_length(descriptions):
    lines = dict_to_list(descriptions)
    return max(len(d.split()) for d in lines)

max_length = max_length(train_descriptions)
print('Maximum sentence length:', max_length)

Maximum sentence length: 34


*Вариант 2*

In [14]:
def create_sequences(tokenizer, max_length, desc_list, photo, vocab_size):
    x1, x2, y = [], [], []
    for desc in desc_list:
        seq = tokenizer.texts_to_sequences([desc])[0]
        for i in range(1, len(seq)):
            input_seq, output_seq = seq[:i], seq[i]
            input_seq = pad_sequences([input_seq], maxlen=max_length)[0]
            output_seq = to_categorical([output_seq], num_classes=vocab_size)[0]
            x1.append(photo)
            x2.append(input_seq)
            y.append(output_seq)
    return np.array(x1), np.array(x2), np.array(y)

def max_length(descriptions):
    lines = dict_to_list(descriptions)
    return max(len(d.split()) for d in lines)

max_length = max_length(train_descriptions)
print('Maximum sentence length:', max_length)

Maximum sentence length: 34


**Defining the Model**

In [15]:
def define_model(vocab_size, max_length):
    # Извлечение особенностей
    inputs1 = Input(shape=(4096,))
    fe1 = Dropout(0.5)(inputs1)
    fe2 = Dense(256, activation='relu')(fe1)
    # Процессор последовательностей
    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
    se2 = Dropout(0.5)(se1)
    se3 = LSTM(256)(se2)
    # Декодер
    decoder1 = Add()([fe2, se3])
    decoder2 = Dense(256, activation='relu')(decoder1)
    outputs = Dense(vocab_size, activation='softmax')(decoder2)
    # Связывание всего в единое
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    # Подведение итогов
    print(model.summary())
    # plot_model(model, to_file='models/model.png', show_shapes=True)
    return model

**Fitting the Model**

In [16]:
x1train, x2train, ytrain = create_sequences(tokenizer, max_length, train_descriptions, train_features, vocab_size)

In [17]:
def data_generator(descriptions, photos, tokenizer, max_length, vocab_size):
    while 1:
        for key, desc_list in descriptions.items():
            photo = photos[key][0]
            in_img, in_seq, out_word = create_sequences(tokenizer, max_length, desc_list, photo, vocab_size)
            yield [in_img, in_seq], out_word

generator = data_generator(train_descriptions, train_features, tokenizer, max_length, vocab_size)
inputs, outputs = next(generator)
print(inputs[0].shape)
print(inputs[1].shape)
print(outputs.shape)


(52, 4096)
(52, 34)
(52, 8109)


In [18]:
model = define_model(vocab_size, max_length)
epochs = 20
steps = len(train_descriptions)
for i in range(epochs):
    generator =  data_generator(train_descriptions, train_features, tokenizer, max_length, vocab_size)
    history = model.fit(generator, epochs=1, steps_per_epoch=steps, verbose=1)
    
    # plt.plot(history.history['accuracy'])
    # plt.plot(history.history['val_accuracy'])
    # plt.title('model accuracy')
    # plt.ylabel('accuracy')
    # plt.xlabel('epoch')
    # plt.legend(['train', 'test'], loc='upper left')
    # plt.show()
    # # summarize history for loss
    # plt.plot(history.history['loss'])
    # plt.plot(history.history['val_loss'])
    # plt.title('model loss')
    # plt.ylabel('loss')
    # plt.xlabel('epoch')
    # plt.legend(['train', 'test'], loc='upper left')
    # plt.show()
    
    model.save('models/model_' + str(i+1) + '.h5')

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 34)]         0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 4096)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, 34, 256)      2075904     ['input_4[0][0]']                
                                                                                                  
 dropout (Dropout)              (None, 4096)         0           ['input_3[0][0]']                
                                                                                            

KeyboardInterrupt: 

In [19]:
from nltk.translate.bleu_score import corpus_bleu


def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

def generate_desc(model, tokenizer, photo, max_length):
    input_text  = 'startseq'
    for i in range(max_length):
        seq = tokenizer.texts_to_sequences([input_text])[0]
        seq = pad_sequences([seq], maxlen=max_length)
        yhat = model.predict([photo, seq], verbose=0)
        yhat = np.argmax(yhat)
        word = word_for_id(yhat, tokenizer)
        if word is None:
            break
        input_text += ' ' + word
        if word == 'endseq':
            break
    return input_text


In [ ]:
from tensorflow.keras.models import load_model
from pickle import load


def evaluate_model(model, descriptions, photos, tokenizer, max_length):
    actual, predicted = [], []
    for key, desc_list in descriptions.items():
        yhat = generate_desc(model, tokenizer, photos[key], max_length)
        references = [d.split() for d in desc_list]
        actual.append(references)
        predicted.append(yhat.split())
    print('BLEU-1:', corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
    print('BLEU-2:', corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
    print('BLEU-3:', corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
    print('BLEU-4:', corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))
    
testImages = 'testImages.txt'
test = load_set(testImages)
print('Dataset:', len(test))
test_descriptions = load_clean_descriptions('descriptions.txt', test)
print('Descriptions: test=', len(test_descriptions))
test_features = load_photo_features('features.p', test)
print('Photos: test=', len(test_features))
tokenizer = load(open('tokenizer.p', 'rb'))
model = load_model('models/model_1.h5')
# max_length = 20

evaluate_model(model, test_descriptions, test_features, tokenizer, max_length)

Dataset: 2091
Descriptions: test= 2091
Photos: test= 2091
